In [80]:
import tensorflow as tf
import json
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

fileTest = open("./Model/DatabaseIA.json")
dataTest = json.load(fileTest)


In [81]:

df = pd.read_json("./Model/DatabaseIA.json")
X = df.loc[:, df.columns != 'Livre_Prefere']
Y = df.loc[:, 'Livre_Prefere']

enc = OneHotEncoder()
X_encoded = enc.fit_transform(X.astype(str)).toarray()

unique_labels = np.unique(Y)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
Y = np.array([label_to_index[label] for label in Y])

# Conversion des valeurs numériques en encodage one-hot
Y = tf.keras.utils.to_categorical(Y)


In [92]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(12, input_shape=(X_encoded.shape[1],), activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(44, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_encoded, Y, epochs=100)



Epoch 1/100
16/16 [==============================] - 1s 2ms/step - loss: 0.6929 - accuracy: 0.0300
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6732 - accuracy: 0.0300
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6523 - accuracy: 0.0320
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6275 - accuracy: 0.0300
Epoch 5/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5968 - accuracy: 0.0280
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5592 - accuracy: 0.0280
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5139 - accuracy: 0.0280
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4606 - accuracy: 0.0280
Epoch 9/100
16/16 [==============================] - 0s 2ms/step - loss: 0.3986 - accuracy: 0.0300
Epoch 10/100
16/16 [==============================] - 0s 2ms/step - loss: 0.3294 - accuracy: 0.0220
Epoch 11/

In [95]:
_, accuracy = model.evaluate(X_encoded, Y)
print('Accuracy: %.2f' % (accuracy*100))

16/16 [==============================] - 0s 2ms/step - loss: 0.0909 - accuracy: 0.1780
Accuracy: 17.80


In [100]:
df_train = df.sample(frac=0.8)
df_test = df.drop(df_train.index)
df_train.head()

,Nom,Prenom,Age,Sexe,Livre_Prefere,Genre_Prefere
318,Bond,James,45,Homme,The Communist Manifesto,POLITICAL SCIENCE
307,Solo,Mr.,51,Femme,A Game of Thrones,Fantasy
221,Skywalker,Mr.,65,Homme,Brave New World,Dystopian
152,Kirk,Harry,35,Femme,The Shining,Horror
204,Potter,Harry,69,Femme,The Selfish Gene,SCIENCE


In [101]:
X_train = df_train.loc[:, df.columns != 'Livre_Prefere']
Y_train = df_train.loc[:, 'Livre_Prefere']
X_test = df_test.loc[:, df.columns != 'Livre_Prefere']
Y_test = df_test.loc[:, 'Livre_Prefere']

In [104]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(12, input_shape=(X_encoded.shape[1],), activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(44, activation='sigmoid'))

tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=True, show_layer_names=False, show_layer_activations=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
